In [ ]:
!pip install tensorflow
!pip install tensorflow-gpu
!pip install keras

In [ ]:
print(tf.__version__)
print(keras.__version__)

In [2]:
!wget -cq https://raw.githubusercontent.com/D-Bhatta/Data-Cleaning-Beginner/master/anne-bonner-tutorial/my_data.csv

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found")
print(f'Found GPU at: {device_name}')